In [2]:
conda install bs4 requests lxml

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import re
import dateutil
import pandas as pd

In [4]:
url = "https://shanghai.douban.com/events/future-1403"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'}
result = requests.get(url, headers = headers)
print(result)

<Response [200]>


In [5]:
src = result.content
soup = BeautifulSoup(src,'html.parser')
events = soup.find_all("li",attrs={"class":"list-entry"})
print(events)

[<li class="list-entry" itemscope="" itemtype="http://data-vocabulary.org/Event">
<div class="pic">
<a href="https://www.douban.com/event/33876311/">
<img data-lazy="https://img9.doubanio.com/pview/event_poster/median/public/bf0f66830932e76.jpg" itemprop="photo" src="https://img9.doubanio.com/f/shire/a1fdee122b95748d81cee426d717c05b5174fe96/pics/blank.gif" width="110"/>
</a>
</div>
<div class="info">
<div class="title">
<a href="https://www.douban.com/event/33876311/" itemprop="url" title="【上海摄影约拍】和行走过二十五个国家的摄影师约拍一套影像集吧">
<span itemprop="summary">【上海摄影约拍】和行走过二十五个国家的摄影师约拍一套影像集吧</span>
</a>
</div>
<p class="event-cate-tag hidden-xs">
<a href="https://www.douban.com/event/search?loc=shanghai&amp;search_text=%E4%B8%8A%E6%B5%B7%E6%91%84%E5%BD%B1%E5%86%99%E7%9C%9F%E7%BA%A6%E6%8B%8D&amp;p=future">上海摄影写真约拍</a> 
            <a href="https://www.douban.com/event/search?loc=shanghai&amp;search_text=%E4%B8%8A%E6%B5%B7%E6%91%84%E5%BD%B1%E5%B8%88&amp;p=future">上海摄影师</a> 
            <a href="https:/

In [6]:
titles = []
urls = []
fees = []
times = []
owners = []
for item in events:
    title = item.find("span",attrs={"itemprop":"summary"})
    url = item.a.get("href")
    fee = item.strong
    time = item.find("li",attrs={"class":"event-time"})
    owner = item.find("a",attrs={"target":"db-event-owner"})
    if title != None and url != None and fee != None and time != None and owner != None:
        time = time.text.strip('时间：\n\n ')
        titles.append(title.text)
        urls.append(url)
        fees.append(fee.text)
        times.append(time)
        owners.append(owner.text)
#print(titles)
#print(urls)
#print(fees)
#print(times)
#print(owners)

df=pd.DataFrame({"Activity Name":titles,"Activity URL":urls, "Fee":fees,"Time Period":times,"Photographer":owners})
df

,Activity Name,Activity URL,Fee,Time Period,Photographer
0,【上海摄影约拍】和行走过二十五个国家的摄影师约拍一套影像集吧,https://www.douban.com/event/33876311/,1元(费用请咨询),2021年08月20日 ~ 2021年11月18日 每天 00:00 - 23:30,摄影师法淦
1,和我一起记录这美美的秋天吧^_^,https://www.douban.com/event/34532995/,1元(具体费用请咨询),2021年10月08日 ~ 2021年10月31日 每天 18:30 - 22:00,HPPJJJ
2,来上海体验一场难忘的胶片摄影之旅,https://www.douban.com/event/34416001/,298元(胶片相机体验)起,2021年08月27日 ~ 2021年11月08日 每天 09:00 - 11:00 14:...,陳小偉HenryC
3,短视频拍摄体验,https://www.douban.com/event/34462678/,免费,08月06日 周五 00:00-12:00 ...,自由自在
4,“美学慢步”CityWalk，逛逛艺术展扫扫街景,https://www.douban.com/event/34514472/,免费,09月19日 周日 13:00-18:00 ...,ugsaggigga
5,上海模特约拍,https://www.douban.com/event/34516117/,160元(每人),2021年09月19日 ~ 2021年10月31日 每天 14:00 - 17:00,wxid
6,♟美少女探店看展 冲冲冲！,https://www.douban.com/event/34518419/,免费,2021年10月08日 ~ 2021年10月31日 每天 19:00 - 23:30,HPPJJJ
7,约拍｜没什么，就是可以把你拍的挺不一样,https://www.douban.com/event/34461931/,200元(定金),2021年08月23日 ~ 2021年11月09日 每天 10:00 - 19:00,lulu
8,上海摄影约拍 | 留住每一个动人瞬间，找到时光里的自己！,https://www.douban.com/event/34471597/,1元(请提前咨询),2021年08月26日 ~ 2021年11月24日 每天 00:00 - 23:30,画画的Franco
9,国风雅集,https://www.douban.com/event/34475658/,80.0元(人均),2021年08月27日 ~ 2021年10月31日 每天 11:00 - 18:00,清如


#### But there seems to be some missing part(...) in the time period, so I tried another way to get the time information.

In [8]:
titles = []
urls = []
fees = []
startdates = []
enddates = []
owners = []
for item in events:
    title = item.find("span",attrs={"itemprop":"summary"})
    url = item.a.get("href")
    fee = item.strong
    startdate = item.find("time",attrs={"itemprop":"startDate"})
    enddate = item.find("time",attrs={"itemprop":"endDate"})
    owner = item.find("a",attrs={"target":"db-event-owner"})
    if title != None and url != None and fee != None and startdate != None and enddate != None and owner != None:
        titles.append(title.text)
        urls.append(url)
        fees.append(fee.text)
        startdates.append(startdate.get("datetime").replace("T","-"))
        enddates.append(enddate.get("datetime").replace("T","-"))
        owners.append(owner.text)
#print(titles)
#print(urls)
#print(fees)
#print(startdates)
#print(enddates)
#print(owners)

df=pd.DataFrame({"Activity Name":titles,"Activity URL":urls, "Fee":fees,"Start Date":startdates,"End Date":enddates,"Photographer":owners})
df.to_csv('Douban Photographer.csv', sep='\t', encoding='utf-8')
df.head()

,Activity Name,Activity URL,Fee,Start Date,End Date,Photographer
0,【上海摄影约拍】和行走过二十五个国家的摄影师约拍一套影像集吧,https://www.douban.com/event/33876311/,1元(费用请咨询),2021-08-20-00:00:00,2021-11-18-23:30:00,摄影师法淦
1,和我一起记录这美美的秋天吧^_^,https://www.douban.com/event/34532995/,1元(具体费用请咨询),2021-10-08-18:30:00,2021-10-31-22:00:00,HPPJJJ
2,来上海体验一场难忘的胶片摄影之旅,https://www.douban.com/event/34416001/,298元(胶片相机体验)起,2021-08-27-09:00:00,2021-11-08-16:00:00,陳小偉HenryC
3,短视频拍摄体验,https://www.douban.com/event/34462678/,免费,2021-08-06-00:00:00,2021-10-16-12:00:00,自由自在
4,“美学慢步”CityWalk，逛逛艺术展扫扫街景,https://www.douban.com/event/34514472/,免费,2021-09-19-13:00:00,2021-12-18-18:00:00,ugsaggigga


#### Now let me try automate Douban search using Selenium. Next time I can combine Selenium and Beautiful Soup for scraping.

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

project_path = os.path.abspath(os.path.dirname('__file__'))
driver_path = os.path.join(project_path, "chromedriver")
driver = webdriver.Chrome(driver_path)
driver.get('https://shanghai.douban.com/')
searchPhrase = driver.find_element_by_id('inp-query')
searchPhrase.send_keys('约拍')
searchBtn = driver.find_element_by_class_name('inp-btn')
searchBtn.click()